In [1]:
import duckdb as db

In [7]:
bancos = db.read_csv('./stage/bancos/EnquadramentoInicia_v2_cleaned.csv')
empregados_cnpj = db.read_csv('./stage/empregados/glassdoor_consolidado_join_match_less_v2_cleaned.csv', delimiter =';')
empregados_seg = db.read_csv('./stage/empregados/glassdoor_consolidado_join_match_v2_cleaned.csv', delimiter = ';')
reclamacoes_2101 = db.read_csv('./stage/reclamacoes/2021_tri_01_cleaned.csv', delimiter = ';')
reclamacoes_2102 = db.read_csv('./stage/reclamacoes/2021_tri_02_cleaned.csv', delimiter = ';')
reclamacoes_2103 = db.read_csv('./stage/reclamacoes/2021_tri_03_cleaned.csv', delimiter = ';')
reclamacoes_2104 = db.read_csv('./stage/reclamacoes/2021_tri_04_cleaned.csv', delimiter = ';')
reclamacoes_2201 = db.read_csv('./stage/reclamacoes/2021_tri_02_cleaned.csv', delimiter = ';')
reclamacoes_2203 = db.read_csv('./stage/reclamacoes/2021_tri_02_cleaned.csv', delimiter = ';')
reclamacoes_2204 = db.read_csv('./stage/reclamacoes/2021_tri_02_cleaned.csv', delimiter = ';')

┌──────────────────────┬───────────────┬───────────────┬───┬──────────┬──────────────────────┬──────────────────┐
│    employer_name     │ reviews_count │ culture_count │ … │   CNPJ   │         nome         │ match_percent,,, │
│       varchar        │     int64     │     int64     │   │  int64   │       varchar        │      int64       │
├──────────────────────┼───────────────┼───────────────┼───┼──────────┼──────────────────────┼──────────────────┤
│ Apex Group           │           450 │           281 │ … │   655522 │ APE POUPEX           │               70 │
│ Banco Alfa           │           175 │            74 │ … │ 58160789 │ SAFRA                │               67 │
│ China Construction…  │           871 │           194 │ … │  7450604 │ CHINA CONSTRUCTION…  │               77 │
│ Mercedes-Benz Mobi…  │           282 │            90 │ … │ 60814191 │ MERCEDES-BENZ        │               73 │
│ Sumitomo Mitsui Fi…  │           903 │           246 │ … │ 60518222 │ BANCO SUMITOMO M

In [18]:
bancos2 = db.sql("select cnpj :: varchar as cnpj, nome, segmento from bancos")
empregados = db.sql ("""select 
                     ec.employer_name, ec.reviews_count, ec.culture_count, ec.salaries_count, ec.benefits_count, ec.employer_website, ec.employer_headquarters, ec.employer_founded, ec.employer_industry, ec.employer_revenue, ec.url, ec.geral, ec.cultura_e_valores, ec.diversidade_e_inclusao, ec.qualidade_de_vida, ec.alta_lideranca, ec.remuneracao_e_beneficios, ec.oportunidades_de_carreira, ec.recomendam_para_outras_pessoas_perc, ec.perspectiva_positiva_da_empresa_perc, ec.cnpj :: varchar as cnpj, '' as segmento, ec.nome, ec."match_percent,,," 
                     from empregados_cnpj ec
                      union all
                     select es.employer_name, es.reviews_count, es.culture_count, es.salaries_count, es.benefits_count, es.employer_website, es.employer_headquarters, es.employer_founded, es.employer_industry, es.employer_revenue, es.url, es.geral, es.cultura_e_valores, es.diversidade_e_inclusao, es.qualidade_de_vida, es.alta_lideranca, es.remuneracao_e_beneficios, es.oportunidades_de_carreira, es.recomendam_para_outras_pessoas_perc, es.perspectiva_positiva_da_empresa_perc, '' as cnpj, es.segmento, es.nome, es."match_percent,,," 
                     from empregados_seg es
                    """)
reclamacoes = db.sql(
    """
        select
            * 
        from reclamacoes_2101 
        union all 
        select 
            * 
        from reclamacoes_2102
        union all 
        select 
            *
        from reclamacoes_2103
        union all 
        select 
            *
        from reclamacoes_2104
        union all 
        select 
            * 
        from reclamacoes_2201
        union all 
        select 
            * 
        from reclamacoes_2203
        union all 
        select 
            *
        from reclamacoes_2204
    """
    )
fato = db.sql("select distinct * from bancos2 b join empregados e on e.cnpj = b.cnpj join reclamacoes r on b.cnpj = r.cnpj_if")

print(fato)

┌──────────┬──────────────────────┬──────────┬───┬──────────────────────┬──────────────────────┬─────────┐
│   cnpj   │         nome         │ segmento │ … │ quantidade_de_clie…  │ quantidade_de_clie…  │   ,,    │
│ varchar  │       varchar        │ varchar  │   │       varchar        │       varchar        │ varchar │
├──────────┼──────────────────────┼──────────┼───┼──────────────────────┼──────────────────────┼─────────┤
│ 60518222 │ BANCO SUMITOMO MIT…  │ S3       │ … │ 1023                 │ 126                  │ ,,      │
├──────────┴──────────────────────┴──────────┴───┴──────────────────────┴──────────────────────┴─────────┤
│ 1 rows                                                                            42 columns (6 shown) │
└────────────────────────────────────────────────────────────────────────────────────────────────────────┘

